In [8]:
import requests
import os
import time
from pathlib import Path
import pandas as pd, pyarrow as pa
import glob 
import matplotlib.pyplot as plt
import duckdb
import numpy as np
from typing import Tuple, Dict, Any, List
import csv
import traceback

In [9]:
PARQUET_PATH = "data/cleaned/yellow_tripdata_clean_snappy.parquet"  # ajuste se necessário
OUTPUT_DIR = "analysis_results"

## 3. Benchmark de consultas :
* Receita por zona: TOP 10 zonas de pickup por receita total média
* Padrões temporais: Agregação por mês/semana (COUNT viagens, SUM receita, AVG distância)
* Horário de pico: Ranking de horários mais movimentados usando window functions (RANK/ROW_NUMBER)
* Análise de gorjeta: Taxa de gorjeta média por borough usando CASE WHEN para categorizar
* Viagens longas vs curtas: Comparativo usando CASE para categorizar distâncias

## 4. Para cada consulta escolhida, você deve:
* Implementar em SQL (DuckDB) e validar os resultados inspecionando os dados
* Implementar equivalente em Pandas e verificar se os resultados são consistentes
* Anotar quais consultas falharam no Pandas (memória, tempo, erro)
* Medir tempo de execução rodando cada consulta 5 vezes em cada ferramenta
* Registrar apenas consultas que executaram com sucesso para comparação de performance

### 4.1 Receita por zona: TOP 10 zonas de pickup por receita total média

In [10]:
# ============================================================
# 1️⃣ Configurações gerais
# ============================================================
PARQUET_PATH = "data/cleaned/yellow_tripdata_clean_snappy.parquet"
OUTPUT_DIR = "analysis_results"
os.makedirs(OUTPUT_DIR, exist_ok=True)
REPEATS = 5

# ============================================================
# 2️⃣ Consulta SQL DuckDB
# ============================================================
SQL_QUERY = f"""
SELECT
  pulocationid,
  AVG(total_amount) AS avg_total_amount,
  COUNT(*) AS trips
FROM parquet_scan('{PARQUET_PATH}')
GROUP BY pulocationid
ORDER BY avg_total_amount DESC
LIMIT 10;
"""

# ============================================================
# 3️⃣ Função de medição
# ============================================================
def time_function(func, *args, repeats=5):
    tempos = []
    ultimo_resultado = None
    for i in range(repeats):
        t0 = time.time()
        resultado = func(*args)
        t1 = time.time()
        tempos.append(t1 - t0)
        ultimo_resultado = resultado
    return tempos, ultimo_resultado

# ============================================================
# 4️⃣ Execução no DuckDB
# ============================================================
def duckdb_query():
    return duckdb.sql(SQL_QUERY).df()

print("=== DuckDB ===")
try:
    tempos_duckdb, resultado_duckdb = time_function(duckdb_query, repeats=REPEATS)
    tempo_medio_duckdb = sum(tempos_duckdb) / len(tempos_duckdb)
    print(resultado_duckdb)
    print(f"Tempo médio DuckDB: {tempo_medio_duckdb:.3f}s")
    resultado_duckdb.to_csv(f"{OUTPUT_DIR}/duckdb_top10_receita_media.csv", index=False)
    duckdb_ok = True
except Exception as e:
    print("❌ Erro no DuckDB:", e)
    duckdb_ok = False
    resultado_duckdb = None

# ============================================================
# 5️⃣ Execução em Pandas
# ============================================================
def pandas_query(parquet_path):
    df = pd.read_parquet(parquet_path, columns=["pulocationid", "total_amount"], engine="pyarrow")
    df = df.dropna(subset=["pulocationid", "total_amount"])
    df["pulocationid"] = df["pulocationid"].astype("Int32")
    res = (
        df.groupby("pulocationid", observed=True)
          .agg(avg_total_amount=("total_amount", "mean"), trips=("total_amount", "size"))
          .reset_index()
          .sort_values("avg_total_amount", ascending=False)
          .head(10)
    )
    return res

print("\n=== Pandas ===")
try:
    tempos_pandas, resultado_pandas = time_function(pandas_query, PARQUET_PATH, repeats=REPEATS)
    tempo_medio_pandas = sum(tempos_pandas) / len(tempos_pandas)
    print(resultado_pandas)
    print(f"Tempo médio Pandas: {tempo_medio_pandas:.3f}s")
    resultado_pandas.to_csv(f"{OUTPUT_DIR}/pandas_top10_receita_media.csv", index=False)
    pandas_ok = True
except Exception as e:
    print("❌ Erro no Pandas:", e)
    pandas_ok = False
    resultado_pandas = None

# ============================================================
# 6️⃣ Validação dos resultados
# ============================================================
if duckdb_ok and pandas_ok:
    iguais = (
        resultado_duckdb["pulocationid"].equals(resultado_pandas["pulocationid"])
        and resultado_duckdb["avg_total_amount"].round(2).equals(resultado_pandas["avg_total_amount"].round(2))
    )
    print("\n=== Validação ===")
    print("Resultados consistentes?", "✅ Sim" if iguais else "⚠️ Pequenas diferenças")

# ============================================================
# 7️⃣ Comparação de performance
# ============================================================
comparacao = pd.DataFrame([
    {
        "Consulta": "Top 10 zonas por receita média",
        "Ferramenta": "DuckDB",
        "Tempo Médio (s)": round(tempo_medio_duckdb, 3) if duckdb_ok else None,
        "Sucesso": duckdb_ok,
    },
    {
        "Consulta": "Top 10 zonas por receita média",
        "Ferramenta": "Pandas",
        "Tempo Médio (s)": round(tempo_medio_pandas, 3) if pandas_ok else None,
        "Sucesso": pandas_ok,
    }
])

print("\n=== Comparação Final ===")
print(comparacao)

comparacao.to_csv(f"{OUTPUT_DIR}/comparacao_receita_media.csv", index=False)
print(f"\nArquivos salvos em: {OUTPUT_DIR}")


=== DuckDB ===
   pulocationid  avg_total_amount    trips
0           265        116.108339    20203
1             1        108.539808     4792
2             5         90.499637      386
3           176         89.067273       11
4           109         85.566000       30
5            99         82.264545       11
6            84         81.630000       14
7           204         81.624444       18
8           132         80.758698  1926652
9           172         80.355926       27
Tempo médio DuckDB: 0.072s

=== Pandas ===
     pulocationid  avg_total_amount    trips
261           265        116.108339    20203
0               1        108.539808     4792
4               5         90.499637      386
172           176         89.067273       11
106           109         85.566000       30
98             99         82.264545       11
83             84         81.630000       14
200           204         81.624444       18
128           132         80.758698  1926652
168           172  

### 4.2 Padrões temporais: Agregação por mês/semana (COUNT viagens, SUM receita, AVG distância)

In [11]:

df = pd.read_parquet(PARQUET_PATH, engine="pyarrow")

# ============================================================
# 1️⃣ Verificar colunas necessárias
# ============================================================

required_cols = ['tpep_pickup_datetime', 'trip_distance', 'total_amount']
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"Colunas faltando no dataframe: {missing}")

# ============================================================
# 2️⃣ Consulta em DuckDB
# ============================================================
duckdb.register("trips", df)

sql_query = """
SELECT
    DATE_TRUNC('month', tpep_pickup_datetime) AS month,
    DATE_TRUNC('week', tpep_pickup_datetime) AS week,
    COUNT(*) AS total_viagens,
    SUM(total_amount) AS receita_total,
    AVG(trip_distance) AS distancia_media
FROM trips
GROUP BY 1, 2
ORDER BY 1
"""

tempos_duckdb = []
resultado_duckdb = None

for i in range(5):
    t0 = time.time()
    resultado_duckdb = duckdb.sql(sql_query).df()
    t1 = time.time()
    tempos_duckdb.append(t1 - t0)

tempo_medio_duckdb = sum(tempos_duckdb) / len(tempos_duckdb)

print(f"\n=== DuckDB ===")
print(resultado_duckdb.head())
print(f"Tempo médio DuckDB: {tempo_medio_duckdb:.3f}s")

# ============================================================
# 3️⃣ Consulta equivalente em Pandas
# ============================================================
tempos_pandas = []
resultado_pandas = None
pandas_ok = True
erro_pandas = None

try:
    for i in range(5):
        t0 = time.time()
        resultado_pandas = (
            df
            .assign(
                month=df["tpep_pickup_datetime"].dt.to_period("M").dt.to_timestamp(),
                week=df["tpep_pickup_datetime"].dt.to_period("W").dt.start_time
            )
            .groupby(["month", "week"], as_index=False)
            .agg(
                total_viagens=("tpep_pickup_datetime", "count"),
                receita_total=("total_amount", "sum"),
                distancia_media=("trip_distance", "mean")
            )
            .sort_values("month")
        )
        t1 = time.time()
        tempos_pandas.append(t1 - t0)

    tempo_medio_pandas = sum(tempos_pandas) / len(tempos_pandas)

    print(f"\n=== Pandas ===")
    print(resultado_pandas.head())
    print(f"Tempo médio Pandas: {tempo_medio_pandas:.3f}s")

except Exception as e:
    pandas_ok = False
    erro_pandas = str(e)
    print("\n❌ Erro ao executar com Pandas:", erro_pandas)

# ============================================================
# 4️⃣ Validação de consistência
# ============================================================
if pandas_ok:
    try:
        diff_viagens = abs(resultado_duckdb["total_viagens"].sum() - resultado_pandas["total_viagens"].sum())
        diff_receita = abs(resultado_duckdb["receita_total"].sum() - resultado_pandas["receita_total"].sum())

        print("\n=== Validação ===")
        print(f"Diferença total_viagens: {diff_viagens}")
        print(f"Diferença receita_total: {diff_receita}")
    except Exception as e:
        print("⚠️ Não foi possível comparar resultados:", e)

# ============================================================
# 5️⃣ Comparação final
# ============================================================
comparacao = pd.DataFrame([
    {
        "Consulta": "Padrões temporais (mês/semana)",
        "Ferramenta": "DuckDB",
        "Tempo Médio (s)": round(tempo_medio_duckdb, 3),
        "Sucesso": True, #Se chegar aqui vai ter dado certo mesmo 
        
    },
    {
        "Consulta": "Padrões temporais (mês/semana)",
        "Ferramenta": "Pandas",
        "Tempo Médio (s)": round(tempo_medio_pandas, 3) if pandas_ok else None,
        "Sucesso": pandas_ok,
        
    }
])

print("\n=== Comparação Final ===")
print(comparacao)

# (opcional) salvar resultados
comparacao.to_csv("analysis_results/comparacao_padroes_temporais.csv", index=False)



=== DuckDB ===
       month       week  total_viagens  receita_total  distancia_media
0 2023-01-01 2023-01-23         693696   1.857369e+07         3.236062
1 2023-01-01 2023-01-02         610212   1.715389e+07         3.669232
2 2023-01-01 2023-01-30         175517   4.824456e+06         3.333357
3 2023-01-01 2023-01-09         688707   1.881083e+07         3.399218
4 2023-01-01 2023-01-16         678519   1.854441e+07         3.391944
Tempo médio DuckDB: 0.311s

=== Pandas ===
       month       week  total_viagens  receita_total  distancia_media
0 2023-01-01 2022-12-26          71176     2208901.82         5.188594
1 2023-01-01 2023-01-02         610212    17153888.42         3.669232
2 2023-01-01 2023-01-09         688707    18810826.76         3.399218
3 2023-01-01 2023-01-16         678519    18544410.30         3.391944
4 2023-01-01 2023-01-23         693696    18573693.40         3.236062
Tempo médio Pandas: 12.673s

=== Validação ===
Diferença total_viagens: 0
Diferença recei

### 4.3 Horário de pico: Ranking de horários mais movimentados usando window functions (RANK/ROW_NUMBER)

In [12]:
df = pd.read_parquet(PARQUET_PATH, engine="pyarrow")

# ============================================================
# 1️⃣ Verificar colunas necessárias
# ============================================================
required_cols = ["tpep_pickup_datetime"]
missing = [c for c in required_cols if c not in df.columns]
if missing:
    raise ValueError(f"Colunas faltando no dataframe: {missing}")

# ============================================================
# 2️⃣ Consulta em DuckDB
# ============================================================
duckdb.register("trips", df)

sql_query = """
WITH hourly_stats AS (
    SELECT
        DATE_TRUNC('hour', tpep_pickup_datetime) AS pickup_hour,
        COUNT(*) AS total_viagens
    FROM trips
    GROUP BY 1
)
SELECT
    pickup_hour,
    total_viagens,
    RANK() OVER (ORDER BY total_viagens DESC) AS rank_viagens
FROM hourly_stats
ORDER BY total_viagens DESC
LIMIT 10;
"""

tempos_duckdb = []
resultado_duckdb = None

for i in range(5):
    t0 = time.time()
    resultado_duckdb = duckdb.sql(sql_query).df()
    t1 = time.time()
    tempos_duckdb.append(t1 - t0)

tempo_medio_duckdb = sum(tempos_duckdb) / len(tempos_duckdb)

print(f"\n=== DuckDB ===")
print(resultado_duckdb)
print(f"Tempo médio DuckDB: {tempo_medio_duckdb:.3f}s")

# ============================================================
# 3️⃣ Consulta equivalente em Pandas
# ============================================================
tempos_pandas = []
resultado_pandas = None
pandas_ok = True
erro_pandas = None

try:
    for i in range(5):
        t0 = time.time()

        resultado_pandas = (
            df.assign(
                pickup_hour=df["tpep_pickup_datetime"].dt.floor("h")
            )
            .groupby("pickup_hour", as_index=False)
            .agg(total_viagens=("tpep_pickup_datetime", "count"))
            .sort_values("total_viagens", ascending=False)
            .reset_index(drop=True)
        )
        # Adiciona rank (equivalente ao RANK() do SQL)
        resultado_pandas["rank_viagens"] = (
            resultado_pandas["total_viagens"]
            .rank(method="dense", ascending=False)
            .astype(int)
        )

        # Pega top 10
        resultado_pandas = resultado_pandas.head(10)

        t1 = time.time()
        tempos_pandas.append(t1 - t0)

    tempo_medio_pandas = sum(tempos_pandas) / len(tempos_pandas)

    print(f"\n=== Pandas ===")
    print(resultado_pandas)
    print(f"Tempo médio Pandas: {tempo_medio_pandas:.3f}s")

except Exception as e:
    pandas_ok = False
    erro_pandas = str(e)
    print("\n❌ Erro ao executar com Pandas:", erro_pandas)

# ============================================================
# 4️⃣ Validação dos resultados
# ============================================================
if pandas_ok:
    try:
        iguais = (
            resultado_duckdb["pickup_hour"].equals(resultado_pandas["pickup_hour"])
            and resultado_duckdb["total_viagens"].equals(resultado_pandas["total_viagens"])
        )
        print("\n=== Validação ===")
        print("Resultados consistentes entre DuckDB e Pandas?" , "✅ Sim" if iguais else "⚠️ Pequenas diferenças")
    except Exception as e:
        print("⚠️ Não foi possível comparar resultados:", e)

# ============================================================
# 5️⃣ Comparação de performance
# ============================================================
comparacao = pd.DataFrame([
    {
        "Consulta": "Horário de pico (rank por hora)",
        "Ferramenta": "DuckDB",
        "Tempo Médio (s)": round(tempo_medio_duckdb, 3),
        "Sucesso": True,
        
    },
    {
        "Consulta": "Horário de pico (rank por hora)",
        "Ferramenta": "Pandas",
        "Tempo Médio (s)": round(tempo_medio_pandas, 3) if pandas_ok else None,
        "Sucesso": pandas_ok,
       
    }
])

print("\n=== Comparação Final ===")
print(comparacao)

# (opcional) salvar resultados
comparacao.to_csv(f"{OUTPUT_DIR}/comparacao_horario_pico.csv", index=False)


=== DuckDB ===
          pickup_hour  total_viagens  rank_viagens
0 2023-06-02 18:00:00          10368             1
1 2023-05-24 19:00:00          10177             2
2 2023-12-01 18:00:00          10105             3
3 2023-09-28 17:00:00           9966             4
4 2023-12-14 18:00:00           9914             5
5 2023-05-11 18:00:00           9805             6
6 2023-10-26 18:00:00           9481             7
7 2023-11-16 18:00:00           9439             8
8 2023-11-02 18:00:00           9413             9
9 2023-05-17 18:00:00           9348            10
Tempo médio DuckDB: 0.279s

=== Pandas ===
          pickup_hour  total_viagens  rank_viagens
0 2023-06-02 18:00:00          10368             1
1 2023-05-24 19:00:00          10177             2
2 2023-12-01 18:00:00          10105             3
3 2023-09-28 17:00:00           9966             4
4 2023-12-14 18:00:00           9914             5
5 2023-05-11 18:00:00           9805             6
6 2023-10-26 18:00:00  

### 4.3 Análise de gorjeta: Taxa de gorjeta média por borough usando CASE WHEN para categorizar

In [13]:
REPEATS = 5

# ============================================================
# 2️⃣ Consulta SQL DuckDB
# ============================================================
# Usamos CASE WHEN para categorizar as zonas por borough (regiões de NYC)
# As faixas são fictícias para fins de exemplo, já que o mapeamento oficial
# viria de um arquivo "taxi_zone_lookup.csv"
SQL_QUERY = f"""
SELECT
  CASE
    WHEN pulocationid BETWEEN 1 AND 63 THEN 'Manhattan'
    WHEN pulocationid BETWEEN 64 AND 128 THEN 'Bronx'
    WHEN pulocationid BETWEEN 129 AND 195 THEN 'Brooklyn'
    WHEN pulocationid BETWEEN 196 AND 231 THEN 'Queens'
    WHEN pulocationid BETWEEN 232 AND 265 THEN 'Staten Island'
    ELSE 'Other'
  END AS borough,
  AVG(CASE WHEN total_amount > 0 THEN tip_amount / total_amount ELSE 0 END) AS avg_tip_rate,
  COUNT(*) AS trips
FROM parquet_scan('{PARQUET_PATH}')
WHERE total_amount > 0 AND tip_amount >= 0
GROUP BY borough
ORDER BY avg_tip_rate DESC;
"""

# ============================================================
# 3️⃣ Função para medir tempo de execução
# ============================================================
def time_function(func, *args, repeats=5):
    tempos = []
    ultimo_resultado = None
    for i in range(repeats):
        t0 = time.time()
        resultado = func(*args)
        t1 = time.time()
        tempos.append(t1 - t0)
        ultimo_resultado = resultado
    return tempos, ultimo_resultado

# ============================================================
# 4️⃣ Execução no DuckDB
# ============================================================
def duckdb_query():
    return duckdb.sql(SQL_QUERY).df()

print("=== DuckDB ===")
try:
    tempos_duckdb, resultado_duckdb = time_function(duckdb_query, repeats=REPEATS)
    tempo_medio_duckdb = sum(tempos_duckdb) / len(tempos_duckdb)
    print(resultado_duckdb)
    print(f"Tempo médio DuckDB: {tempo_medio_duckdb:.3f}s")
    resultado_duckdb.to_csv(f"{OUTPUT_DIR}/duckdb_tiprate_borough.csv", index=False)
    duckdb_ok = True
except Exception as e:
    print("❌ Erro no DuckDB:", e)
    duckdb_ok = False
    resultado_duckdb = None

# ============================================================
# 5️⃣ Execução em Pandas
# ============================================================
def pandas_query(parquet_path):
    df = pd.read_parquet(parquet_path, columns=["pulocationid", "tip_amount", "total_amount"], engine="pyarrow")
    df = df[(df["total_amount"] > 0) & (df["tip_amount"] >= 0)]

    # Categorizar manualmente os boroughs
    def categorize_zone(z):
        if 1 <= z <= 63:
            return "Manhattan"
        elif 64 <= z <= 128:
            return "Bronx"
        elif 129 <= z <= 195:
            return "Brooklyn"
        elif 196 <= z <= 231:
            return "Queens"
        elif 232 <= z <= 265:
            return "Staten Island"
        else:
            return "Other"

    df["borough"] = df["pulocationid"].apply(categorize_zone)
    df["tip_rate"] = df["tip_amount"] / df["total_amount"]

    result = (
        df.groupby("borough", observed=True)
          .agg(avg_tip_rate=("tip_rate", "mean"), trips=("tip_rate", "size"))
          .reset_index()
          .sort_values("avg_tip_rate", ascending=False)
    )
    return result

print("\n=== Pandas ===")
try:
    tempos_pandas, resultado_pandas = time_function(pandas_query, PARQUET_PATH, repeats=REPEATS)
    tempo_medio_pandas = sum(tempos_pandas) / len(tempos_pandas)
    print(resultado_pandas)
    print(f"Tempo médio Pandas: {tempo_medio_pandas:.3f}s")
    resultado_pandas.to_csv(f"{OUTPUT_DIR}/pandas_tiprate_borough.csv", index=False)
    pandas_ok = True
except Exception as e:
    print("❌ Erro no Pandas:", e)
    pandas_ok = False
    resultado_pandas = None

# ============================================================
# 6️⃣ Validação dos resultados
# ============================================================
if duckdb_ok and pandas_ok:
    iguais = set(resultado_duckdb["borough"]) == set(resultado_pandas["borough"])
    print("\n=== Validação ===")
    print("Boroughs idênticos?", "✅ Sim" if iguais else "⚠️ Diferenças detectadas")

# ============================================================
# 7️⃣ Comparação de performance
# ============================================================
comparacao = pd.DataFrame([
    {
        "Consulta": "Taxa média de gorjeta por borough",
        "Ferramenta": "DuckDB",
        "Tempo Médio (s)": round(tempo_medio_duckdb, 3) if duckdb_ok else None,
        "Sucesso": duckdb_ok,
    },
    {
        "Consulta": "Taxa média de gorjeta por borough",
        "Ferramenta": "Pandas",
        "Tempo Médio (s)": round(tempo_medio_pandas, 3) if pandas_ok else None,
        "Sucesso": pandas_ok,
    }
])

print("\n=== Comparação Final ===")
print(comparacao)

comparacao.to_csv(f"{OUTPUT_DIR}/comparacao_tiprate_borough.csv", index=False)
print(f"\nArquivos salvos em: {OUTPUT_DIR}")

=== DuckDB ===
         borough  avg_tip_rate     trips
0  Staten Island      0.124277   9513802
1          Bronx      0.119510   5743284
2       Brooklyn      0.118797  15519458
3         Queens      0.117416   2834779
4      Manhattan      0.113917   2412939
Tempo médio DuckDB: 0.469s

=== Pandas ===
         borough  avg_tip_rate     trips
4  Staten Island      0.124277   9513802
0          Bronx      0.119510   5743284
1       Brooklyn      0.118797  15519458
3         Queens      0.117416   2834779
2      Manhattan      0.113917   2412939
Tempo médio Pandas: 12.465s

=== Validação ===
Boroughs idênticos? ✅ Sim

=== Comparação Final ===
                            Consulta Ferramenta  Tempo Médio (s)  Sucesso
0  Taxa média de gorjeta por borough     DuckDB            0.469     True
1  Taxa média de gorjeta por borough     Pandas           12.465     True

Arquivos salvos em: analysis_results
